### Useful import

In [2]:
import pandas as pd
import numpy as np
import glob
import re
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import tree
import random

In [3]:
path_shots_LNA = "Data/Team/LNA_Regular_Season/shots/*"
path_shots_LNB = "Data/Team/LNB_Regular_Season/shots/*"
df_shots = []


for file in glob.iglob(path_shots_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_shots.append(frame)

# for file in glob.iglob(path_shots_LNB):
#     frame = pd.read_csv(file,sep=';')
#     frame['Year'] = file[35:39]
#     df_shots.append(frame)
    
df_shots = pd.concat(df_shots)

df_shots.drop(df_shots.columns[[0]], axis=1, inplace=True)
df_shots.reset_index(inplace=True)
df_shots = df_shots.drop(df_shots[df_shots.Team == 'Durchschnitt'].index)
df_shots.set_index(['Team','Year'], inplace=True)
df_shots.sort_index(inplace=True)
df_shots

index  GP   GF    SOG     S%  SOG/GP
Team                      Year                                      
EHC Biel                  0809     11  50  126  1'126  11.19      23
                          0910      8  50  134  1'340  10.00      27
                          1011      4  50  135  1'530   8.82      31
                          1112      4  50  114  1'584   7.20      32
                          1213      2  50  160  1'733   9.23      35
                          1314      0  50  115  1'760   6.53      35
                          1415      4  50  142  1'547   9.18      31
                          1516      0  50  128  1'741   7.35      35
EV Zug                    0809      3  50  171  1'524  11.22      30
                          0910      0  50  165  1'881   8.77      38
                          1011      1  50  176  1'701  10.35      34
                          1112      1  50  173  1'623  10.66      32
                          1213     10  50  172  1'515  11.35      30
                          1314      1  50  132  1'721   7.67      34
                          1415      3  50  166  1'569  10.58      31
                          1516      3  50  161  1'667   9.66      33
Fribourg-Gottéron         0809      2  50  154  1'550   9.94      31
                          0910      1  50  145  1'735   8.36      35
                          1011      5  50  157  1'507  10.42      30
                          1112      3  50  156  1'586   9.84      32
                          1213      5  50  163  1'568  10.40      31
                          1314      3  50  151  1'616   9.34      32
                          1415      0  50  144  1'630   8.83      33
                          1516     12  50  148  1'399  10.58      28
Genève-Servette HC        0809     12  50  157    942  16.67      19
                          0910     12  50  166  1'066  15.57      21
                          1011     10  50  129  1'250  10.32      25
                          1112      5  50  117  1'519   7.70      30
                          1213      9  50  146  1'540   9.48      31
                          1314      5  50  158  1'558  10.14      31
...                               ...  ..  ...    ...    ...     ...
Lausanne HC               1516      7  50  123  1'553   7.92      31
SC Bern                   0809      5  50  187  1'459  12.82      29
                          0910      3  50  154  1'558   9.88      31
                          1011      0  50  160  1'755   9.12      35
                          1112      0  50  153  1'632   9.38      33
                          1213      0  50  169  1'792   9.43      36
                          1314      8  50  126  1'497   8.42      30
                          1415      9  50  158  1'489  10.61      30
                          1516      2  50  152  1'702   8.93      34
SC Rapperswil-Jona Lakers 0809     10  50  143  1'325  10.79      26
                          0910     10  50  129  1'244  10.37      25
                          1011     11  50  138  1'199  11.51      24
                          1112     11  50   99  1'250   7.92      25
                          1213     11  50  139  1'505   9.24      30
                          1314      4  50  117  1'559   7.50      31
                          1415      1  50  121  1'577   7.67      32
SCL Tigers                0809      4  50  170  1'497  11.36      30
                          0910      9  50  141  1'277  11.04      26
                          1011      6  50  149  1'482  10.05      30
                          1112      6  50  124  1'508   8.22      30
                          1213      6  50  117  1'552   7.54      31
                          1516     11  50  135  1'436   9.40      29
ZSC Lions                 0809      0  50  176  1'671  10.53      33
                          0910      2  50  185  1'593  11.61      32
                          1011      8  50  132  1'411   9.36      28
                          1112

In [4]:
path_powerkill_LNA = "Data/Team/LNA_Regular_Season/powerkill/*"
path_powerkill_LNB = "Data/Team/LNB_Regular_Season/powerkill/*"

df_powerkill = []

for file in glob.iglob(path_powerkill_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_powerkill.append(frame)

# for file in glob.iglob(path_powerkill_LNB):
#     frame = pd.read_csv(file,sep=';')
#     frame['Year'] = file[39:43]
#     df_powerkill.append(frame)
    
df_powerkill = pd.concat(df_powerkill)

df_powerkill.drop(df_powerkill.columns[[0]], axis=1, inplace=True)
df_powerkill.reset_index(inplace=True)
df_powerkill = df_powerkill.drop(df_powerkill[df_powerkill.Team == 'Durchschnitt'].index)
df_powerkill.set_index(['Team','Year'], inplace=True)
df_powerkill.sort_index(inplace=True)

df_powerkill.head(10)

index  GP  PK SI  PK SI/GP  PK GA  PK GA/GP    PK%     PKT  \
Team     Year                                                               
EHC Biel 0809      7  50    306      6.12     55      1.10  82.03  461:34   
         0910      4  50    266      5.32     45      0.90  83.08  405:36   
         1011     11  50    201      4.02     47      0.94  76.62  323:29   
         1112      9  50    200      4.00     35      0.70  82.50  329:47   
         1213      4  50    198      3.96     33      0.66  83.33  317:33   
         1314     10  50    197      3.94     34      0.68  82.74  298:08   
         1415     12  50    167      3.34     37      0.74  77.84  261:24   
         1516     12  50    197      3.94     53      1.06  73.10  283:07   
EV Zug   0809      4  50    289      5.78     46      0.92  84.08  453:07   
         0910      2  50    229      4.58     34      0.68  85.15  370:40   

              PK time/GP  SHG  PK GF/GP  
Team     Year                            
EHC Biel 0809      09:13    3      0.06  
         0910      08:06    7      0.14  
         1011      06:28    3      0.06  
         1112      06:35    1      0.02  
         1213      06:21    7      0.14  
         1314      05:57    5      0.10  
         1415      05:13    3      0.06  
         1516      05:39    3      0.06  
EV Zug   0809      09:03    8      0.16  
         0910      07:24    6      0.12

In [5]:
path_goals_LNA = "Data/Team/LNA_Regular_Season/goals/*"
path_goals_LNB = "Data/Team/LNB_Regular_Season/goals/*"

df_goals = []

for file in glob.iglob(path_goals_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_goals.append(frame)

# for file in glob.iglob(path_goals_LNB):
#     frame = pd.read_csv(file,sep=';')
#     frame['Year'] = file[35:39]
#     df_goals.append(frame)
    
df_goals = pd.concat(df_goals)

df_goals.drop(df_goals.columns[[0]], axis=1, inplace=True)
df_goals.reset_index(inplace=True)
df_goals = df_goals.drop(df_goals[df_goals.Team == 'Durchschnitt'].index)
df_goals.set_index(['Team','Year'], inplace=True)
df_goals.sort_index(inplace=True)

df_goals.head(10)

index  GP   GF  GF/GP   GA  GA/GP  GF even  GF even/GP  \
Team     Year                                                           
EHC Biel 0809      0  50  126   2.52  213   4.26       94        1.88   
         0910     10  50  134   2.68  182   3.64       80        1.60   
         1011      8  50  135   2.70  173   3.46       89        1.78   
         1112     10  50  114   2.28  128   2.56       80        1.60   
         1213      5  50  160   3.20  161   3.22      112        2.24   
         1314     11  50  115   2.30  154   3.08       74        1.48   
         1415      7  50  142   2.84  164   3.28      101        2.02   
         1516     11  50  128   2.56  175   3.50       79        1.58   
EV Zug   0809      1  50  171   3.42  166   3.32      114        2.28   
         0910      3  50  165   3.30  125   2.50      101        2.02   

               GA even  GA even/GP  PPG  PPG/GP  PP GA  PP GA/GP  SHG  \
Team     Year                                                           
EHC Biel 0809      145        2.90   29    0.58     13      0.26    3   
         0910      129        2.58   44    0.88      5      0.10    7   
         1011      116        2.32   39    0.78      6      0.12    3   
         1112       88        1.76   30    0.60      4      0.08    1   
         1213      118        2.36   37    0.74      7      0.14    7   
         1314      110        2.20   30    0.60      8      0.16    5   
         1415      122        2.44   35    0.70      3      0.06    3   
         1516      115        2.30   44    0.88      5      0.10    3   
EV Zug   0809      112        2.24   49    0.98      8      0.16    8   
         0910       86        1.72   50    1.00      4      0.08    6   

               PK GF/GP  PK GA  PK GA/GP  SOWGF  SOWGA  
Team     Year                                           
EHC Biel 0809      0.06     55      1.10      0      0  
         0910      0.14     45      0.90      3      3  
         1011      0.06     47      0.94      4      4  
         1112      0.02     35      0.70      3      1  
         1213      0.14     33      0.66      4      3  
         1314      0.10     34      0.68      6      2  
         1415      0.06     37      0.74      3      2  
         1516      0.06     53      1.06      2      2  
EV Zug   0809      0.16     46      0.92      0      0  
         0910      0.12     34      0.68      3      1

In [6]:
path_penalties_LNA = "Data/Team/LNA_Regular_Season/penalties/*"
path_penalties_LNB = "Data/Team/LNB_Regular_Season/penalties/*"

df_penalties = []

for file in glob.iglob(path_penalties_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_penalties.append(frame)

# for file in glob.iglob(path_penalties_LNB):
#     frame = pd.read_csv(file,sep=';')
#     frame['Year'] = file[39:43]
#     df_penalties.append(frame)
    
df_penalties = pd.concat(df_penalties)

df_penalties.drop(df_penalties.columns[[0]], axis=1, inplace=True)
df_penalties.reset_index(inplace=True)
df_penalties = df_penalties.drop(df_penalties[df_penalties.Team == 'Durchschnitt'].index)
df_penalties.set_index(['Team','Year'], inplace=True)
df_penalties.sort_index(inplace=True)

df_penalties.head(10)

index  GP  2'   5'  10'  GM  MP  PIM Total
Team     Year                                            
EHC Biel 0809      9  50  321   2    8   4   0        812
         0910      3  50  303   2   12   1   1        776
         1011      8  50  226   4    8   4   0        632
         1112      8  50  216   6    2   5   1        602
         1213      7  50  239   1    8   1   0        583
         1314     12  50  214   0    2   1   0        468
         1415     10  50  189   2    5   3   0        498
         1516      9  50  203   4    3   4   0        536
EV Zug   0809      5  50  317   6    7   7   0        874
         0910      4  50  289   3   12   3   0        773

In [7]:
path_powerplay_LNA = "Data/Team/LNA_Regular_Season/powerplay/*"
path_powerplay_LNB = "Data/Team/LNB_Regular_Season/powerplay/*"

df_powerplay = []

for file in glob.iglob(path_powerplay_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_powerplay.append(frame)

# for file in glob.iglob(path_powerplay_LNB):
#     frame = pd.read_csv(file,sep=';')
#     frame['Year'] = file[39:43]
#     df_powerplay.append(frame)
    

df_powerplay = pd.concat(df_powerplay)

df_powerplay.drop(df_powerplay.columns[[0]], axis=1, inplace=True)
df_powerplay.reset_index(inplace=True)
df_powerplay = df_powerplay.drop(df_powerplay[df_powerplay.Team == 'Durchschnitt'].index)
df_powerplay.set_index(['Team','Year'], inplace=True)
df_powerplay.sort_index(inplace=True)

df_powerplay.head(10)

index  GP  PP OP  PP OP/GP  PPG  PPG/GP    PP%     PPT  \
Team     Year                                                           
EHC Biel 0809     12  50    261      5.22   29    0.58  11.11  431:52   
         0910      9  50    259      5.18   44    0.88  16.99  398:40   
         1011      1  50    201      4.02   39    0.78  19.40  323:04   
         1112      8  50    198      3.96   30    0.60  15.15  323:53   
         1213      9  50    231      4.62   37    0.74  16.02  363:29   
         1314     10  50    197      3.94   30    0.60  15.23  304:24   
         1415      5  50    188      3.76   35    0.70  18.62  305:57   
         1516      5  50    200      4.00   44    0.88  22.00  311:08   
EV Zug   0809      8  50    290      5.80   49    0.98  16.90  444:04   
         0910      0  50    221      4.42   50    1.00  22.62  342:16   

              PP time/GP  PP GA  PP GA/GP  
Team     Year                              
EHC Biel 0809      08:38     13      0.26  
         0910      07:58      5      0.10  
         1011      06:27      6      0.12  
         1112      06:28      4      0.08  
         1213      07:16      7      0.14  
         1314      06:05      8      0.16  
         1415      06:07      3      0.06  
         1516      06:13      5      0.10  
EV Zug   0809      08:52      8      0.16  
         0910      06:50      4      0.08

In [8]:
def timer_to_int(time) :
    m = re.search('(\d+):(\d+)', time)
    return int(m.group(1))*60 + int(m.group(2))

In [31]:
df = pd.concat([df_shots, df_powerkill, df_goals, df_penalties, df_powerplay], axis=1)
df = df.loc[:,~df.columns.duplicated()]
df = df.reset_index().set_index(['Team','Year'])
df['Number of year'] = 1
df.drop(['GP', 'index', 'SOG', 'S%', 'SOG/GP'],axis=1, inplace=True)
#df.drop(['Neuchâtel'], axis=0, inplace=True)
#df['SOG'] = df['SOG'].apply(lambda x : 0 if x == 0 else int(x[0] + x[2:]))
df['PKT'] = df['PKT'].apply(lambda x : timer_to_int(x))
df['PK time/GP'] = df['PK time/GP'].apply(lambda x : timer_to_int(x))
df['PPT'] = df['PPT'].apply(lambda x : timer_to_int(x))
df['PP time/GP'] = df['PP time/GP'].apply(lambda x : timer_to_int(x))
df.ix[0:5,0:8]
#df.to_csv('Data/Cleaned_Data/team_regular_season.csv')

GF  PK SI  PK SI/GP  PK GA  PK GA/GP    PK%    PKT  PK time/GP
Team     Year                                                                 
EHC Biel 0809  126    306      6.12     55      1.10  82.03  27694         553
         0910  134    266      5.32     45      0.90  83.08  24336         486
         1011  135    201      4.02     47      0.94  76.62  19409         388
         1112  114    200      4.00     35      0.70  82.50  19787         395
         1213  160    198      3.96     33      0.66  83.33  19053         381

In [13]:
df = df.astype(float)

In [20]:
PO_path_LNA = 'Data/Team/Rankings/by_match/LNA_playoff/*'
PO_path_LNB = 'Data/Team/Rankings/by_match/LNB_playoff/*'

df_match = []

for file in glob.iglob(PO_path_LNA):
    f = pd.read_csv(file, ';')
    f['Year'] = file[40:44]
    df_match.append(f)

# for file in glob.iglob(PO_path_LNB):
#     f = pd.read_csv(file, ';')
#     f['Year'] = file[40:44]
#     df_match.append(f)


df_match = pd.concat(df_match)
df_match = df_match[['Home', 'Away', 'Resultat', 'Year']]
df_match['Result'] = [ 'h' if x[1] > x[3] else 'a' for x in df_match["Resultat"]]
#df_match['Series_ID'] = df_match['Year']+'_'+set([df_match['Home'], df_match['Away']])
#df_match['Series_ID'] = [0]*len(df_match)

In [27]:
df_match['Series_ID'] = df_match.apply(lambda x: x.Year + '_' + ' '.join(sorted([x.Home, x.Away])), axis=1)
df_match['Opponent 0'] = df_match.apply(lambda x: sorted([x.Home, x.Away])[0], axis=1)
df_match['Opponent 1'] = df_match.apply(lambda x: sorted([x.Home, x.Away])[1], axis=1)
df_match['Result'] = [0 if (x == 'h' and y == a) or (x == 'a' and z == a) else 1 
                 for x, y, z, a in zip(df_match['Result'], df_match['Home'], df_match['Away'], df_match['Opponent 0'])]
df_match[df_match['Year'] == '1516'].head(3)

,Home,Away,Resultat,Year,Result,Series_ID,Opponent 0,Opponent 1
0,HC Lugano,SC Bern,5:4,1516,1,1516_HC Lugano SC Bern,HC Lugano,SC Bern
1,SC Bern,HC Lugano,1:0,1516,1,1516_HC Lugano SC Bern,HC Lugano,SC Bern
2,HC Lugano,SC Bern,2:3,1516,1,1516_HC Lugano SC Bern,HC Lugano,SC Bern


In [22]:
play_off_series = df_match.groupby('Series_ID').agg({
    'Year': {
        'Year': 'first'
    },
    'Home': {
        'Homefield_advantage': lambda x: x.value_counts().index[0],
    },
    'Opponent 0': {
        'Opponent 0': 'first'
    },
    'Opponent 1': {
        'Opponent 1': 'first'
    },
    'Result': {
        'Winner': lambda x: 1 if x.sum() >= 4 else 0
    }
})
play_off_series.columns = play_off_series.columns.droplevel()
play_off_series.head(5)

,Winner,Year,Opponent 1,Homefield_advantage,Opponent 0
Series_ID,,,,,
0809_EV Zug Kloten Flyers,1,0809,Kloten Flyers,Kloten Flyers,EV Zug
0809_EV Zug SC Bern,0,0809,SC Bern,SC Bern,EV Zug
0809_Fribourg-Gottéron HC Davos,1,0809,HC Davos,HC Davos,Fribourg-Gottéron
0809_Fribourg-Gottéron ZSC Lions,0,0809,ZSC Lions,ZSC Lions,Fribourg-Gottéron
0809_Genève-Servette HC Kloten Flyers,1,0809,Kloten Flyers,Kloten Flyers,Genève-Servette HC


In [23]:
#for r in play_off_series.iterrows() :
#    rand = random.random()
#    if (rand<=3/8):
#        temp = play_off_series.loc[r[0],'Opponent 0']
#        play_off_series.loc[r[0],'Opponent 0'] = play_off_series.loc[r[0],'Opponent 1']
#        play_off_series.loc[r[0],'Opponent 1'] = temp
#        play_off_series.loc[r[0],'Winner'] = (r[1]['Winner'] + 1) % 2



In [36]:
df_match_stats = []

for x in play_off_series.iterrows() :
    year = x[1]['Year']
    opp0 = x[1]['Opponent 0']
    opp1 = x[1]['Opponent 1']
    winner = x[1]['Winner']
    
#    line = df.loc[(opp0,year)]-df.loc[(opp1,year)] if winner == opp0 else df.loc[(opp1,year)]-df.loc[(opp0,year)]
    line = df.loc[(opp0,year)]-df.loc[(opp1,year)]
    aa = pd.DataFrame(line).transpose()
    aa['Series_ID'] = year + '_' + ' '.join(sorted([opp0, opp1]))
    aa.set_index(['Series_ID'],inplace=True)

    df_match_stats = pd.concat([pd.DataFrame(df_match_stats),aa])

cc = pd.concat([play_off_series,df_match_stats],axis=1)
cc['Homefield_advantage'] = cc['Homefield_advantage'] == cc['Opponent 1']
cc.Homefield_advantage = cc.Homefield_advantage.astype(int)

cc.to_csv('Data/Ready_for_ML/lna_playoff.csv')
cc.drop('Homefield_advantage', axis=1).ix[0:5,0:9]

,Winner,Year,Opponent 1,Opponent 0,GF,PK SI,PK SI/GP,PK GA,PK GA/GP
Series_ID,,,,,,,,,
0809_EV Zug Kloten Flyers,1,0809,Kloten Flyers,EV Zug,-3.0,18.0,0.36,4.0,0.08
0809_EV Zug SC Bern,0,0809,SC Bern,EV Zug,-16.0,27.0,0.54,-5.0,-0.10
0809_Fribourg-Gottéron HC Davos,1,0809,HC Davos,Fribourg-Gottéron,-30.0,-25.0,-0.50,11.0,0.22
0809_Fribourg-Gottéron ZSC Lions,0,0809,ZSC Lions,Fribourg-Gottéron,-22.0,31.0,0.62,17.0,0.34
0809_Genève-Servette HC Kloten Flyers,1,0809,Kloten Flyers,Genève-Servette HC,-17.0,26.0,0.52,0.0,0.00
